In [1]:
!git clone https://github.com/kamalkraj/BERT-NER.git

Cloning into 'BERT-NER'...
remote: Enumerating objects: 246, done.
remote: Total 246 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (246/246), 1.67 MiB | 20.37 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [2]:
cd BERT-NER/

/content/BERT-NER


In [3]:
!wget https://www.dropbox.com/s/x0uhua7h4bcfvsf/out_ner-20200923T154226Z-001.zip

--2020-09-23 15:56:51--  https://www.dropbox.com/s/x0uhua7h4bcfvsf/out_ner-20200923T154226Z-001.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/x0uhua7h4bcfvsf/out_ner-20200923T154226Z-001.zip [following]
--2020-09-23 15:56:51--  https://www.dropbox.com/s/raw/x0uhua7h4bcfvsf/out_ner-20200923T154226Z-001.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc53d410b6310d9f03a50169edf3.dl.dropboxusercontent.com/cd/0/inline/A___zzRB3lc2sXfdCHufaPx-q9-55Km-91sjOGPpX4PHO24YL4qYlMD4PLkSV64kaDOj-GupCFyJSlYT685z0QSxWi7dqRWRKrR40issdMhdThNJOZfkyoMhgJn0IO3rT08/file# [following]
--2020-09-23 15:56:51--  https://uc53d410b6310d9f03a50169edf3.dl.dropboxusercontent.com/cd/0/inline/A___zzRB3lc2sXfdCHufaPx-q9-55Km-91sjOGPpX4PHO24YL4qYlM

In [4]:
!unzip out_ner-20200923T154226Z-001.zip

Archive:  out_ner-20200923T154226Z-001.zip
  inflating: out_ner/eval_results.txt  
  inflating: out_ner/model_config.json  
  inflating: out_ner/vocab.txt       
  inflating: out_ner/added_tokens.json  
  inflating: out_ner/special_tokens_map.json  
  inflating: out_ner/config.json     
  inflating: out_ner/tokenizer_config.json  
  inflating: out_ner/pytorch_model.bin  


In [5]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 184kB 4.7MB/s 
     |████████████████████████████████| 748.9MB 15kB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 1.5MB 53.9MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449907 sha256=20696d00e1c0b4e24cbc827e54085ec507664a7f2083e35ef95dc6ffee50825e
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=4f5062607d77a49b661923124e6c6dc260c5448f03c0af9cb801d94f84bd4e27
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built nltk sacremoses
ERROR: torchvision 0.7.0+cu101 has requirement torch

### **Fine-Tuning:**
> For finetuning or training the **bert-base** model run the 'run_ner.py' as command given below.

> In below command we have to pass different arguments:
   
*   '--data_dir' argument required to collect dataset. Pass 'data/' as argument which we can see as directory inside 'BERT-NER' folder for the previous comment and command for 'BERT-NER files' .
*   '--bert_model' used to download **pretrained bert base** model of Hugging Face transformers. There are different model-names as suggested by hugging face for argument, here we select 'bert-base-cased'.
*   '--task_name' argument used for task to perform. Enter 'ner' as we will train the model for Named Entity Recogintion(NER).
*   '--output_dir' argument is for where to store fine-tuned model. We give name 'out_base' for directory  where fine-tuned model stored.
*   Other arguments like '--max_seq_length', '--num_train_epochs' and '--warmup_proportion', just give values as suggested in repository.
*   For training pass argument '--do_train' and after that evaluating for results pass argument '--do_eval'.





    

In [ ]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_ner --max_seq_length=128 --do_train --num_train_epochs 3 --do_eval --warmup_proportion=0.1

In [6]:
%%writefile bert.py
"""BERT NER Inference."""

from __future__ import absolute_import, division, print_function

import json
import os

import torch
import torch.nn.functional as F
from nltk import word_tokenize
from pytorch_transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)


class BertNer(BertForTokenClassification):

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, valid_ids=None):
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask, head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32,device='cuda' if torch.cuda.is_available() else 'cpu')
        for i in range(batch_size):
            jj = -1
            for j in range(max_len):
                    if valid_ids[i][j].item() == 1:
                        jj += 1
                        valid_output[i][jj] = sequence_output[i][j]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)
        return logits

class Ner:

    def __init__(self,model_dir: str):
        self.model , self.tokenizer, self.model_config = self.load_model(model_dir)
        self.label_map = self.model_config["label_map"]
        self.max_seq_length = self.model_config["max_seq_length"]
        self.label_map = {int(k):v for k,v in self.label_map.items()}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = self.model.to(self.device)
        self.model.eval()

    def load_model(self, model_dir: str, model_config: str = "model_config.json"):
        model_config = os.path.join(model_dir,model_config)
        model_config = json.load(open(model_config))
        model = BertNer.from_pretrained(model_dir)
        tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=model_config["do_lower"])
        return model, tokenizer, model_config

    def tokenize(self, text: str):
        """ tokenize input"""
        words = word_tokenize(text)
        tokens = []
        valid_positions = []
        for i,word in enumerate(words):
            token = self.tokenizer.tokenize(word)
            tokens.extend(token)
            for i in range(len(token)):
                if i == 0:
                    valid_positions.append(1)
                else:
                    valid_positions.append(0)
        return tokens, valid_positions

    def preprocess(self, text: str):
        """ preprocess """
        tokens, valid_positions = self.tokenize(text)
        ## insert "[CLS]"
        tokens.insert(0,"[CLS]")
        valid_positions.insert(0,1)
        ## insert "[SEP]"
        tokens.append("[SEP]")
        valid_positions.append(1)
        segment_ids = []
        for i in range(len(tokens)):
            segment_ids.append(0)
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        while len(input_ids) < self.max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            valid_positions.append(0)
        return input_ids,input_mask,segment_ids,valid_positions

    def predict(self, text: str):
        input_ids,input_mask,segment_ids,valid_ids = self.preprocess(text)
        input_ids = torch.tensor([input_ids],dtype=torch.long,device=self.device)
        input_mask = torch.tensor([input_mask],dtype=torch.long,device=self.device)
        segment_ids = torch.tensor([segment_ids],dtype=torch.long,device=self.device)
        valid_ids = torch.tensor([valid_ids],dtype=torch.long,device=self.device)
        with torch.no_grad():
            logits = self.model(input_ids, segment_ids, input_mask,valid_ids)
        logits = F.softmax(logits,dim=2)
        logits_label = torch.argmax(logits,dim=2)
        logits_label = logits_label.detach().cpu().numpy().tolist()[0]

        logits_confidence = [values[label].item() for values,label in zip(logits[0],logits_label)]

        logits = []
        pos = 0
        for index,mask in enumerate(valid_ids[0]):
            if index == 0:
                continue
            if mask == 1:
                logits.append((logits_label[index-pos],logits_confidence[index-pos]))
            else:
                pos += 1
        logits.pop()

        labels = [(self.label_map[label],confidence) for label,confidence in logits]
        words = word_tokenize(text)
        assert len(labels) == len(words)

        Person = []
        Location = []
        Organization = []
        Miscelleneous = []

        for word, (label, confidence) in zip(words, labels):
            if label=="B-PER" or label=="I-PER":
                Person.append(word)
            elif label=="B-LOC" or label=="I-LOC":
                Location.append(word)
            elif label=="B-ORG" or label=="I-ORG":
                Organization.append(word)
            elif label=="B-MISC" or label=="I-MISC":
                Miscelleneous.append(word)
            else:
                output = None

        output = []
        for word, (label, confidence) in zip(words, labels):      
            if label == "B-PER":
                output.append(' '.join(Person) + ": Person")
            if label=="B-LOC":
                output.append(' '.join(Location) + ": Location")
            if label=="B-MISC":
                output.append(' '.join(Miscelleneous) + ": Miscelleneous Entity")
            if label=="B-ORG":
                output.append(' '.join(Organization) + ": Organization")
                
        return output



Overwriting bert.py


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from bert import Ner
model = Ner("out_ner/")

text= "Russia made a deepfake about the US president Donald Trump working as a reporter after losing elections."
print("Text to predict Entity:", text)

output = model.predict(text)
for prediction in output:
    print(prediction)

Text to predict Entity: Russia made a deepfake about the US president Donald Trump working as a reporter after losing elections.
Russia US: Location
Russia US: Location
Donald Trump: Person
